In [1]:
import sys
import os
project_path = os.path.abspath("..")
sys.path.append(project_path)

In [2]:
from utilities.grover_state_preparation import *
from utilities.auxiliaries import *
from config import *

import numpy as np
import pandas as pd
from scipy.stats import beta, wasserstein_distance
from scipy.optimize import minimize
from itertools import combinations
from joblib import Parallel, delayed
from tqdm import tqdm  

In [3]:
p_i_set = beta.pdf(np.linspace(a, b, 2**m), alpha_, beta_)
p_i_set /= p_i_set.sum()
n_angles = len(get_grover_angles(p_i_set, m))

In [4]:
num_cores = max(1, os.cpu_count() - 0) # adjust how many cores to deplete

In [ ]:
def optimize_configuration(idx_thetas_to_optimize):
    """
    Optimize a selected subset of Grover state-preparation angles and evaluate
    the resulting generative performance.
    This function is used in the to process more then 1 combination of angles at a time.
    Use this only for testing purposes.

    Parameters
    ----------
    idx_thetas_to_optimize : iterable of int
        Indices of the angles to be treated as trainable parameters during
        optimization.

    Returns
    -------
    dict
        A dictionary containing:
        - "trainable angles": the indices of optimized parameters;
        - "best loss": the minimum optimization loss achieved over multiple runs;
        - "Earth Mover's Distance": the Wasserstein distance between the
          generated and target distributions obtained from the optimized circuit.
    """

    idx_thetas_to_optimize = list(idx_thetas_to_optimize)

    thetas_to_optimize = np.array(generate_parameters(len(idx_thetas_to_optimize), k=2))
    thetas = get_grover_angles(p_i_set, m)

    best_loss = float('inf')
    best_thetas = None

    for run in range(runs):
        result = minimize(objective_function, thetas_to_optimize,
                          args=(idx_thetas_to_optimize, thetas, p_i_set, shots),
                          method=optimizer_type, options={"disp": False, "maxiter": max_iterations})

        current_loss = result.fun
        thetas_to_optimize = result.x

        if current_loss < best_loss:
            best_loss = current_loss
            best_thetas = np.array(thetas.copy())
            best_thetas[idx_thetas_to_optimize] = thetas_to_optimize 

    final_qc  = state_expansion(m, best_thetas)
    counts = backend.run(transpile(final_qc, backend=backend), shots=shots).result().get_counts(final_qc)

    samples = np.array([counts.get(state, 0) for state in all_states], dtype=float)
    samples /= samples.sum()

    x = np.linspace(a, b, 2**m)

    emd = wasserstein_distance(x, x, u_weights=samples, v_weights=p_i_set) 

    return {
        "trainable angles": str(idx_thetas_to_optimize),
        "best loss": best_loss,
        "Earth Mover's Distance": emd
    } 

In [ ]:
if __name__ == "__main__":
    print("Preparing configurations...")
    
    configurations = [comb for k in range(1, n_angles + 1) for comb in combinations(range(n_angles), k)]
    print(f"Done generating {len(configurations)} configurations.")

    results_list = Parallel(n_jobs=num_cores)(
        delayed(optimize_configuration)(idx_thetas_to_optimize) for idx_thetas_to_optimize in tqdm(configurations, desc="Processing Configurations")
    )

    results = pd.DataFrame(results_list)

    saving_folder = project_path #change this according to your prefered saving folder
    excel_filename = os.path.join(saving_folder, "testing_m4.csv")  #change this according to your prefered file saving name
    results.to_csv(excel_filename, index=False)

    print(f"\nResults saved at: '{excel_filename}'")

Preparing configurations...
Done generating 20 configurations.


Processing Configurations: 100%|██████████| 20/20 [00:00<00:00, 151.09it/s]



Results saved at: 'c:\Users\marco\Desktop\generative_models_via_sqsp\testing_m4.csv'
